In [22]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go

# Step 1: Fetching AAPL Stock Data from yfinance
aapl = yf.Ticker("AAPL")
data = aapl.history(period="60d", interval="1h")  # Adjust the period as needed


In [23]:

# Step 2: Calculating Bollinger Bands
data['SMA'] = data['Close'].rolling(window=20).mean()
data['SD'] = data['Close'].rolling(window=20).std()
data['UB'] = data['SMA'] + 2 * data['SD']
data['LB'] = data['SMA'] - 2 * data['SD']

# Step 3: Interpreting Bollinger Bands

# Step 4: Visualizing Bollinger Bands with Plotly
fig = go.Figure()

# Add the price chart
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Price'))

# Add the Upper Bollinger Band (UB) and shade the area
fig.add_trace(go.Scatter(x=data.index, y=data['UB'], mode='lines', name='Upper Bollinger Band', line=dict(color='red')))
fig.add_trace(go.Scatter(x=data.index, y=data['LB'], fill='tonexty', mode='lines', name='Lower Bollinger Band', line=dict(color='green')))

# Add the Middle Bollinger Band (MA)
fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='Middle Bollinger Band', line=dict(color='blue')))

# Customize the chart layout
fig.update_layout(title='AAPL Stock Price with Bollinger Bands',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  showlegend=True)

# Show the chart
fig.show()

In [24]:
def recommend_strategy(data):
    last_row = data.iloc[-1]
    if last_row['Close'] > last_row['UB']:
        return 'Sell'
    elif last_row['Close'] < last_row['LB']:
        return 'Buy'
    else:
        return 'Hold'

In [25]:
# Recommend Strategy
strategy = recommend_strategy(data)
print(f'Recommended Strategy: {strategy}')

Recommended Strategy: Hold


In [31]:
# Importing Required Libraries
import pandas as pd
import yfinance as yf
import cufflinks as cf
import plotly.offline

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Fetching AAPL Stock Data
aapl = yf.Ticker("AAPL")
data = aapl.history(period="60d", interval="1h")

# Calculate True Range and Average True Range
data["HL"] = data["High"] - data["Low"]
data["HC"] = abs(data["High"] - data["Close"].shift())
data["LC"] = abs(data["Low"] - data["Close"].shift())
data["TR"] = data[["HL", "HC", "LC"]].max(axis=1)
data["ATR"] = data["TR"].rolling(window=14).mean()

# Determine Trading Strategy based on ATR
data["PreviousHigh"] = data["High"].shift()
data["PreviousLow"] = data["Low"].shift()

data["BuySignal"] = data["Close"] > data["PreviousHigh"] + data["ATR"]
data["SellSignal"] = data["Close"] < data["PreviousLow"] - data["ATR"]

data["Strategy"] = "Hold"
data.loc[data["BuySignal"], "Strategy"] = "Buy"
data.loc[data["SellSignal"], "Strategy"] = "Sell"

# Plot the chart
data[["Close", "ATR"]].iplot(
    title="Average True Range of AAPL with Trading Signals",
    xTitle="Date",
    yTitle="Price / ATR",
    secondary_y="ATR",
    secondary_y_title="ATR",
    theme="pearl",
)

# Print the recommended strategy
last_row = data.iloc[-1]
print(f'Recommended Strategy: {last_row["Strategy"]}')

Recommended Strategy: Hold
